In [1]:
from neo4j import GraphDatabase, exceptions
from decouple import config

In [2]:
class CriarRelacionamentosNeo4j:

    def __init__(self):
        self.uri = config('NEO4J_URI')
        self.user = config('NEO4J_USERNAME')
        self.password = config('NEO4J_PASSWORD')
        self._driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))

    def close(self):
        self._driver.close()

    def create_relationships(self):
        with self._driver.session() as session:
            try:
                # Relacionamento entre as tabelas Região e UF
                session.run("""
                    MATCH (r:Regiao)
                    MATCH (u:UF)
                    CREATE (r)-[:LOCALIZADA_EM]->(u)
                """)

                # Relacionamento entre as tabelas UF e Município
                session.run("""
                    MATCH (u:UF)
                    MATCH (m:Municipio)
                    CREATE (u)-[:POSSUI]->(m)
                """)

                # Relacionamento entre as tabelas Município e Beneficiário
                session.run("""
                    MATCH (m:Municipio)
                    MATCH (b:Beneficiario)
                    CREATE (m)-[:RESIDE_EM]->(b)
                """)

                # Relacionamento entre as tabelas Beneficiário e IES
                session.run("""
                    MATCH (b:Beneficiario)
                    MATCH (i:IES)
                    CREATE (b)-[:CURSA_NA]->(i)
                """)

                # Relacionamento entre as tabelas IES e Curso
                session.run("""
                    MATCH (i:IES)
                    MATCH (c:Curso)
                    CREATE (i)-[:OFERECE]->(c)
                """)

                # Relacionamento entre as tabelas Bolsa, IES e Curso
                session.run("""
                    MATCH (b:Bolsa)
                    MATCH (i:IES)
                    MATCH (c:Curso)
                    CREATE (b)-[:VINCULADA_A]->(i)
                    CREATE (b)-[:VINCULADA_A]->(c)
                """)
                
                print("Relacionamentos criados com sucesso!")

            except exceptions.TransientError as e:
                if "MemoryPoolOutOfMemoryError" in str(e):
                    print("Erro: Limite de memória atingido. Tente novamente com uma carga de dados menor.")
                else:
                    raise

    def start(self):
        self.create_relationships()


In [3]:
# Exemplo de uso:
relacionamentos_neo4j = CriarRelacionamentosNeo4j()
relacionamentos_neo4j.start()
relacionamentos_neo4j.close()

Erro: Limite de memória atingido. Tente novamente com uma carga de dados menor.
